BENV0093 - Protected Areas Mask
Excludes protected areas from WLC

In [4]:
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from pathlib import Path

print("="*80)
print("PROTECTED AREAS MASK PROCESSING")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
DATASET_DIR = BASE_DIR / 'Dateset_T2'
OUTPUT_DIR = BASE_DIR / 'outputs'

PROTECTED_SHP = DATASET_DIR / 'Protected' / 'protected_hard_constraint.shp'
WLC_TIF = OUTPUT_DIR / 'wlc_suitability_score_5criteria.tif'
PROTECTED_MASK_TIF = OUTPUT_DIR / 'protected_areas_mask.tif'

CRS_EPSG = 27700
NODATA_VALUE = -9999

# Step 1: Load reference grid from WLC
print("\nStep 1: Loading reference grid from WLC")
with rasterio.open(WLC_TIF) as src:
    ref_transform = src.transform
    ref_shape = src.shape
    ref_crs = src.crs
    
    wlc_data = src.read(1)
    land_mask = (wlc_data != NODATA_VALUE) & (~np.isnan(wlc_data))
    
    rows, cols = ref_shape
    pixel_size_x = abs(ref_transform.a)
    pixel_size_y = abs(ref_transform.e)
    
    x_min = ref_transform.c
    x_max = ref_transform.c + cols * pixel_size_x
    y_max = ref_transform.f
    y_min = ref_transform.f + rows * ref_transform.e

print(f"Grid: {rows} x {cols}")
print(f"Pixel size: {pixel_size_x:.0f} x {pixel_size_y:.0f} m")
print(f"Land cells: {land_mask.sum():,}")

# Step 2: Load protected areas
print("\nStep 2: Loading protected areas")
protected = gpd.read_file(PROTECTED_SHP)
print(f"Loaded {len(protected):,} protected areas")

if protected.crs is None:
    protected = protected.set_crs(f'EPSG:{CRS_EPSG}')
elif protected.crs.to_epsg() != CRS_EPSG:
    protected = protected.to_crs(f'EPSG:{CRS_EPSG}')

# Clip to study area
protected_clipped = protected.cx[x_min:x_max, y_min:y_max]
print(f"Clipped to study area: {len(protected_clipped):,} features")

# Calculate area
if len(protected_clipped) > 0:
    protected_area_km2 = protected_clipped.geometry.area.sum() / 1_000_000
    print(f"Protected area: {protected_area_km2:.1f} km²")

# Step 3: Rasterize protected areas
print("\nStep 3: Rasterizing protected areas")

if len(protected_clipped) > 0:
    protected_dissolved = protected_clipped.dissolve()
    
    protected_raster = rasterize(
        [(geom, 1) for geom in protected_dissolved.geometry],
        out_shape=(rows, cols),
        transform=ref_transform,
        fill=0,
        dtype='uint8',
        all_touched=True
    )
    
    protected_raster[~land_mask] = 255
    
    excluded_cells = (protected_raster == 1).sum()
    allowed_cells = (protected_raster == 0).sum()
    
    print(f"Excluded: {excluded_cells:,} cells ({excluded_cells/land_mask.sum()*100:.1f}% of land)")
    print(f"Allowed: {allowed_cells:,} cells ({allowed_cells/land_mask.sum()*100:.1f}% of land)")
else:
    protected_raster = np.zeros((rows, cols), dtype='uint8')
    protected_raster[~land_mask] = 255
    excluded_cells = 0
    allowed_cells = land_mask.sum()
    print("No protected areas found")

# Step 4: Save mask
print("\nStep 4: Saving mask")

mask_output = protected_raster.astype('int16')
mask_output[protected_raster == 255] = NODATA_VALUE

metadata = {
    'driver': 'GTiff',
    'height': rows,
    'width': cols,
    'count': 1,
    'dtype': 'int16',
    'crs': ref_crs,
    'transform': ref_transform,
    'compress': 'lzw',
    'nodata': NODATA_VALUE
}

with rasterio.open(PROTECTED_MASK_TIF, 'w', **metadata) as dst:
    dst.write(mask_output, 1)

print(f"Saved: {PROTECTED_MASK_TIF}")

# Step 5: Verify alignment
print("\nStep 5: Verifying alignment")
with rasterio.open(WLC_TIF) as ref, rasterio.open(PROTECTED_MASK_TIF) as mask:
    checks = {
        'Shape': ref.shape == mask.shape,
        'Transform': ref.transform.almost_equals(mask.transform, precision=0.01),
        'CRS': ref.crs == mask.crs,
    }
    
    for check, result in checks.items():
        status = "MATCH" if result else "MISMATCH"
        print(f"  {check}: {status}")
    
    if all(checks.values()):
        print("\nPERFECT ALIGNMENT")

print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)

print(f"""
Summary:
  Protected areas: {len(protected_clipped):,} features
  Area: {protected_area_km2:.1f} km²
  
  Excluded: {excluded_cells:,} cells ({excluded_cells/land_mask.sum()*100:.1f}% of land)
  Allowed: {allowed_cells:,} cells ({allowed_cells/land_mask.sum()*100:.1f}% of land)
  
  Mask convention:
    0 = Allowed
    1 = Excluded (protected)
    -9999 = NoData (sea)

Output: {PROTECTED_MASK_TIF}
""")

PROTECTED AREAS MASK PROCESSING

Step 1: Loading reference grid from WLC
Grid: 235 x 131
Pixel size: 5000 x 5000 m
Land cells: 9,743

Step 2: Loading protected areas
Loaded 542 protected areas
Clipped to study area: 540 features
Protected area: 34301.5 km²

Step 3: Rasterizing protected areas
Excluded: 2,652 cells (27.2% of land)
Allowed: 7,091 cells (72.8% of land)

Step 4: Saving mask
Saved: D:\BENV0093\outputs\protected_areas_mask.tif

Step 5: Verifying alignment
  Shape: MATCH
  Transform: MATCH
  CRS: MATCH

PERFECT ALIGNMENT

PROCESSING COMPLETE

Summary:
  Protected areas: 540 features
  Area: 34301.5 km²
  
  Excluded: 2,652 cells (27.2% of land)
  Allowed: 7,091 cells (72.8% of land)
  
  Mask convention:
    0 = Allowed
    1 = Excluded (protected)
    -9999 = NoData (sea)

Output: D:\BENV0093\outputs\protected_areas_mask.tif



Settlement Buffer (500m) Mask
Excludes 500m buffer around settlements

In [7]:
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from pathlib import Path

print("="*80)
print("SETTLEMENT BUFFER (500m) MASK PROCESSING")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
DATASET_DIR = BASE_DIR / 'Dateset_T2'
OUTPUT_DIR = BASE_DIR / 'outputs'

BUILT_UP_GPKG = DATASET_DIR / 'buildup' / 'os_open_built_up_areas.gpkg'
WLC_TIF = OUTPUT_DIR / 'wlc_suitability_score_5criteria.tif'
SETTLEMENT_MASK_TIF = OUTPUT_DIR / 'settlement_500m_buffer_mask.tif'

BUFFER_DISTANCE = 500  # meters
CRS_EPSG = 27700
NODATA_VALUE = -9999

print(f"Buffer distance: {BUFFER_DISTANCE}m")

# Step 1: Load reference grid from WLC
print("\nStep 1: Loading reference grid from WLC")
with rasterio.open(WLC_TIF) as src:
    ref_transform = src.transform
    ref_shape = src.shape
    ref_crs = src.crs
    
    wlc_data = src.read(1)
    land_mask = (wlc_data != NODATA_VALUE) & (~np.isnan(wlc_data))
    
    rows, cols = ref_shape
    pixel_size_x = abs(ref_transform.a)
    pixel_size_y = abs(ref_transform.e)
    
    x_min = ref_transform.c
    x_max = ref_transform.c + cols * pixel_size_x
    y_max = ref_transform.f
    y_min = ref_transform.f + rows * ref_transform.e

print(f"Grid: {rows} x {cols}")
print(f"Pixel size: {pixel_size_x:.0f} x {pixel_size_y:.0f} m")
print(f"Land cells: {land_mask.sum():,}")

# Step 2: Load built-up areas
print("\nStep 2: Loading built-up areas")
print("(This may take 1-2 minutes...)")

built_up = gpd.read_file(BUILT_UP_GPKG)
print(f"Loaded {len(built_up):,} built-up areas")

if built_up.crs is None:
    built_up = built_up.set_crs(f'EPSG:{CRS_EPSG}')
elif built_up.crs.to_epsg() != CRS_EPSG:
    print("Reprojecting to EPSG:27700...")
    built_up = built_up.to_crs(f'EPSG:{CRS_EPSG}')

# Clip to study area
print("Clipping to study area...")
built_up_clipped = built_up.cx[x_min:x_max, y_min:y_max]
print(f"Clipped: {len(built_up_clipped):,} features")

if len(built_up_clipped) == 0:
    print("\nWARNING: No built-up areas in study area")
    print("Creating empty mask (all allowed)")
    
    settlement_raster = np.zeros((rows, cols), dtype='uint8')
    settlement_raster[~land_mask] = 255
    excluded_cells = 0
    allowed_cells = land_mask.sum()
    buffered_area = 0
else:
    # Step 3: Apply buffer
    print(f"\nStep 3: Applying {BUFFER_DISTANCE}m buffer")
    print("(This may take 2-5 minutes...)")
    
    built_up_buffered = built_up_clipped.copy()
    built_up_buffered['geometry'] = built_up_clipped.buffer(BUFFER_DISTANCE)
    
    original_area = built_up_clipped.geometry.area.sum() / 1_000_000
    buffered_area = built_up_buffered.geometry.area.sum() / 1_000_000
    
    print(f"Original area: {original_area:.1f} km²")
    print(f"Buffered area: {buffered_area:.1f} km²")
    print(f"Expansion: {buffered_area - original_area:.1f} km²")
    
    # Dissolve overlapping buffers
    print("Dissolving overlapping buffers...")
    built_up_dissolved = built_up_buffered.dissolve()
    print(f"Dissolved into {len(built_up_dissolved)} feature(s)")
    
    # Step 4: Rasterize
    print("\nStep 4: Rasterizing buffered areas")
    print("(This may take 1-2 minutes...)")
    
    settlement_raster = rasterize(
        [(geom, 1) for geom in built_up_dissolved.geometry],
        out_shape=(rows, cols),
        transform=ref_transform,
        fill=0,
        dtype='uint8',
        all_touched=True
    )
    
    settlement_raster[~land_mask] = 255
    
    excluded_cells = (settlement_raster == 1).sum()
    allowed_cells = (settlement_raster == 0).sum()
    
    print(f"Excluded: {excluded_cells:,} cells ({excluded_cells/land_mask.sum()*100:.1f}% of land)")
    print(f"Allowed: {allowed_cells:,} cells ({allowed_cells/land_mask.sum()*100:.1f}% of land)")

# Step 5: Save mask
print("\nStep 5: Saving mask")

mask_output = settlement_raster.astype('int16')
mask_output[settlement_raster == 255] = NODATA_VALUE

metadata = {
    'driver': 'GTiff',
    'height': rows,
    'width': cols,
    'count': 1,
    'dtype': 'int16',
    'crs': ref_crs,
    'transform': ref_transform,
    'compress': 'lzw',
    'nodata': NODATA_VALUE
}

with rasterio.open(SETTLEMENT_MASK_TIF, 'w', **metadata) as dst:
    dst.write(mask_output, 1)

print(f"Saved: {SETTLEMENT_MASK_TIF}")

# Step 6: Verify alignment
print("\nStep 6: Verifying alignment")
with rasterio.open(WLC_TIF) as ref, rasterio.open(SETTLEMENT_MASK_TIF) as mask:
    checks = {
        'Shape': ref.shape == mask.shape,
        'Transform': ref.transform.almost_equals(mask.transform, precision=0.01),
        'CRS': ref.crs == mask.crs,
    }
    
    for check, result in checks.items():
        status = "MATCH" if result else "MISMATCH"
        print(f"  {check}: {status}")
    
    if all(checks.values()):
        print("\nPERFECT ALIGNMENT")

print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)

print(f"""
Summary:
  Built-up areas: {len(built_up_clipped):,} features
  Buffer distance: {BUFFER_DISTANCE}m
  Buffered area: {buffered_area:.1f} km²
  
  Excluded: {excluded_cells:,} cells ({excluded_cells/land_mask.sum()*100:.1f}% of land)
  Allowed: {allowed_cells:,} cells ({allowed_cells/land_mask.sum()*100:.1f}% of land)
  
  Mask convention:
    0 = Allowed
    1 = Excluded (within 500m buffer)
    -9999 = NoData (sea)

Output: {SETTLEMENT_MASK_TIF}
""")

SETTLEMENT BUFFER (500m) MASK PROCESSING
Buffer distance: 500m

Step 1: Loading reference grid from WLC
Grid: 235 x 131
Pixel size: 5000 x 5000 m
Land cells: 9,743

Step 2: Loading built-up areas
(This may take 1-2 minutes...)


C:\Users\888\anaconda3\Lib\site-packages\pyogrio\geopandas.py:275: UserWarning: More than one layer found in 'os_open_built_up_areas.gpkg': 'os_open_built_up_areas' (default), 'os_open_built_up_extents', 'os_open_non_built_up_extents'. Specify layer parameter to avoid this warning.
  result = read_func(


Loaded 8,585 built-up areas
Clipping to study area...
Clipped: 8,585 features

Step 3: Applying 500m buffer
(This may take 2-5 minutes...)
Original area: 16912.6 km²
Buffered area: 55798.5 km²
Expansion: 38886.0 km²
Dissolving overlapping buffers...
Dissolved into 1 feature(s)

Step 4: Rasterizing buffered areas
(This may take 1-2 minutes...)
Excluded: 5,885 cells (60.4% of land)
Allowed: 3,858 cells (39.6% of land)

Step 5: Saving mask
Saved: D:\BENV0093\outputs\settlement_500m_buffer_mask.tif

Step 6: Verifying alignment
  Shape: MATCH
  Transform: MATCH
  CRS: MATCH

PERFECT ALIGNMENT

PROCESSING COMPLETE

Summary:
  Built-up areas: 8,585 features
  Buffer distance: 500m
  Buffered area: 55798.5 km²
  
  Excluded: 5,885 cells (60.4% of land)
  Allowed: 3,858 cells (39.6% of land)
  
  Mask convention:
    0 = Allowed
    1 = Excluded (within 500m buffer)
    -9999 = NoData (sea)

Output: D:\BENV0093\outputs\settlement_500m_buffer_mask.tif



Water Bodies Mask
Extracts water bodies from UK Land Cover Map 2024

In [8]:
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
from pathlib import Path

print("="*80)
print("WATER BODIES MASK PROCESSING")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
DATASET_DIR = BASE_DIR / 'Dateset_T2'
OUTPUT_DIR = BASE_DIR / 'outputs'

LCM_GB_TIF = DATASET_DIR / 'landcover' / 'data' / 'gb2024lcm1km_dominant_target.tif'
WLC_TIF = OUTPUT_DIR / 'wlc_suitability_score_5criteria.tif'
WATER_MASK_TIF = OUTPUT_DIR / 'water_bodies_mask.tif'

WATER_CLASSES = {
    13: 'Saltwater',
    14: 'Freshwater'
}

NODATA_VALUE = -9999

print("\nWater classes to exclude:")
for class_id, name in WATER_CLASSES.items():
    print(f"  {class_id}: {name}")

# Step 1: Load reference grid from WLC
print("\nStep 1: Loading reference grid from WLC")
with rasterio.open(WLC_TIF) as src:
    ref_transform = src.transform
    ref_shape = src.shape
    ref_crs = src.crs
    
    wlc_data = src.read(1)
    land_mask = (wlc_data != NODATA_VALUE) & (~np.isnan(wlc_data))
    
    rows, cols = ref_shape

print(f"Grid: {rows} x {cols}")
print(f"CRS: {ref_crs}")
print(f"Land cells: {land_mask.sum():,}")

# Step 2: Load GB Land Cover Map
print("\nStep 2: Loading GB Land Cover Map")
with rasterio.open(LCM_GB_TIF) as src:
    lcm_gb_data = src.read(1)
    lcm_gb_transform = src.transform
    lcm_gb_crs = src.crs
    lcm_gb_shape = src.shape
    lcm_gb_nodata = src.nodata
    lcm_gb_dtype = src.dtypes[0]
    
    print(f"LCM shape: {lcm_gb_shape}")
    print(f"LCM CRS: {lcm_gb_crs}")
    print(f"LCM dtype: {lcm_gb_dtype}")
    print(f"LCM nodata: {lcm_gb_nodata}")
    
    # Handle nodata
    if lcm_gb_nodata is None or ('uint8' in str(lcm_gb_dtype) and lcm_gb_nodata < 0):
        lcm_gb_nodata = 0
        print(f"Using nodata: {lcm_gb_nodata}")
    
    # Check for water classes
    unique_classes = np.unique(lcm_gb_data[lcm_gb_data != lcm_gb_nodata])
    water_found = [c for c in WATER_CLASSES.keys() if c in unique_classes]
    print(f"Unique classes: {len(unique_classes)}")
    print(f"Water classes found: {water_found}")

# Step 3: Reproject LCM to reference grid
print("\nStep 3: Reprojecting LCM to reference grid")
print(f"From {lcm_gb_shape} (1km) to {ref_shape} (5km)")

lcm_reprojected = np.zeros((rows, cols), dtype='int16')
lcm_reprojected.fill(NODATA_VALUE)

# Convert to int16 for proper nodata handling
if lcm_gb_data.dtype != 'int16':
    lcm_gb_data_int16 = lcm_gb_data.astype('int16')
    if lcm_gb_nodata is not None:
        lcm_gb_data_int16[lcm_gb_data == lcm_gb_nodata] = NODATA_VALUE
    src_nodata = NODATA_VALUE
else:
    lcm_gb_data_int16 = lcm_gb_data
    src_nodata = lcm_gb_nodata if lcm_gb_nodata is not None else NODATA_VALUE

reproject(
    source=lcm_gb_data_int16,
    destination=lcm_reprojected,
    src_transform=lcm_gb_transform,
    src_crs=lcm_gb_crs,
    src_nodata=src_nodata,
    dst_transform=ref_transform,
    dst_crs=ref_crs,
    dst_nodata=NODATA_VALUE,
    resampling=Resampling.nearest
)

print("Reprojection complete")

# Check reprojected classes
unique_reprojected = np.unique(lcm_reprojected[lcm_reprojected != NODATA_VALUE])
water_reprojected = [c for c in WATER_CLASSES.keys() if c in unique_reprojected]
print(f"Water classes after reproject: {water_reprojected}")

# Step 4: Create water mask
print("\nStep 4: Creating water bodies mask")

water_mask = np.zeros((rows, cols), dtype='uint8')

for class_id, class_name in WATER_CLASSES.items():
    class_cells = (lcm_reprojected == class_id).sum()
    if class_cells > 0:
        water_mask[lcm_reprojected == class_id] = 1
        print(f"  Class {class_id} ({class_name}): {class_cells} cells")

water_mask[~land_mask] = 255

excluded_cells = (water_mask == 1).sum()
allowed_cells = (water_mask == 0).sum()

print(f"\nExcluded (water): {excluded_cells:,} cells ({excluded_cells/land_mask.sum()*100:.1f}% of land)")
print(f"Allowed (land): {allowed_cells:,} cells ({allowed_cells/land_mask.sum()*100:.1f}% of land)")

# Step 5: Save mask
print("\nStep 5: Saving mask")

mask_output = water_mask.astype('int16')
mask_output[water_mask == 255] = NODATA_VALUE

metadata = {
    'driver': 'GTiff',
    'height': rows,
    'width': cols,
    'count': 1,
    'dtype': 'int16',
    'crs': ref_crs,
    'transform': ref_transform,
    'compress': 'lzw',
    'nodata': NODATA_VALUE
}

with rasterio.open(WATER_MASK_TIF, 'w', **metadata) as dst:
    dst.write(mask_output, 1)

print(f"Saved: {WATER_MASK_TIF}")

# Step 6: Verify alignment
print("\nStep 6: Verifying alignment")
with rasterio.open(WLC_TIF) as ref, rasterio.open(WATER_MASK_TIF) as mask:
    checks = {
        'Shape': ref.shape == mask.shape,
        'Transform': ref.transform.almost_equals(mask.transform, precision=0.01),
        'CRS': ref.crs == mask.crs,
    }
    
    for check, result in checks.items():
        status = "MATCH" if result else "MISMATCH"
        print(f"  {check}: {status}")
    
    if all(checks.values()):
        print("\nPERFECT ALIGNMENT")

# Calculate area
grid_cell_area = 25  # km² (5km x 5km)
water_area_km2 = excluded_cells * grid_cell_area

print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)

print(f"""
Summary:
  Water classes: Saltwater (13) + Freshwater (14)
  Source: UK Land Cover Map 2024 (1km resolution)
  
  Excluded: {excluded_cells:,} cells ({excluded_cells/land_mask.sum()*100:.1f}% of land)
  Allowed: {allowed_cells:,} cells ({allowed_cells/land_mask.sum()*100:.1f}% of land)
  Water area: ~{water_area_km2:.0f} km²
  
  Mask convention:
    0 = Allowed
    1 = Excluded (water)
    -9999 = NoData (sea)

Output: {WATER_MASK_TIF}
""")

WATER BODIES MASK PROCESSING

Water classes to exclude:
  13: Saltwater
  14: Freshwater

Step 1: Loading reference grid from WLC
Grid: 235 x 131
CRS: EPSG:27700
Land cells: 9,743

Step 2: Loading GB Land Cover Map
LCM shape: (1300, 700)
LCM CRS: EPSG:27700
LCM dtype: uint8
LCM nodata: None
Using nodata: 0
Unique classes: 21
Water classes found: [13, 14]

Step 3: Reprojecting LCM to reference grid
From (1300, 700) (1km) to (235, 131) (5km)
Reprojection complete
Water classes after reproject: [13, 14]

Step 4: Creating water bodies mask
  Class 13 (Saltwater): 105 cells
  Class 14 (Freshwater): 61 cells

Excluded (water): 70 cells (0.7% of land)
Allowed (land): 9,673 cells (99.3% of land)

Step 5: Saving mask
Saved: D:\BENV0093\outputs\water_bodies_mask.tif

Step 6: Verifying alignment
  Shape: MATCH
  Transform: MATCH
  CRS: MATCH

PERFECT ALIGNMENT

PROCESSING COMPLETE

Summary:
  Water classes: Saltwater (13) + Freshwater (14)
  Source: UK Land Cover Map 2024 (1km resolution)
  
  Ex

Combine 3 hard constraints

In [9]:
"""
BENV0093 - Combine Hard Constraints Masks
Combines three constraint masks into one
"""

import numpy as np
import rasterio
from pathlib import Path

print("="*80)
print("COMBINE HARD CONSTRAINTS MASKS")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
OUTPUT_DIR = BASE_DIR / 'outputs'

PROTECTED_MASK_TIF = OUTPUT_DIR / 'protected_areas_mask.tif'
SETTLEMENT_MASK_TIF = OUTPUT_DIR / 'settlement_500m_buffer_mask.tif'
WATER_MASK_TIF = OUTPUT_DIR / 'water_bodies_mask.tif'
WLC_TIF = OUTPUT_DIR / 'wlc_suitability_score_5criteria.tif'

COMBINED_MASK_TIF = OUTPUT_DIR / 'hard_constraints_combined_mask.tif'

NODATA_VALUE = -9999

print("\nCombining three constraint masks:")
print("  1. Protected areas")
print("  2. Settlement buffer (500m)")
print("  3. Water bodies")

# Step 1: Load reference from WLC
print("\nStep 1: Loading reference grid")
with rasterio.open(WLC_TIF) as src:
    ref_meta = src.meta.copy()
    ref_transform = src.transform
    ref_shape = src.shape
    
    wlc_data = src.read(1)
    land_mask = (wlc_data != NODATA_VALUE) & (~np.isnan(wlc_data))
    
    rows, cols = ref_shape

print(f"Grid: {rows} x {cols}")
print(f"Land cells: {land_mask.sum():,}")

# Step 2: Load constraint masks
print("\nStep 2: Loading constraint masks")

with rasterio.open(PROTECTED_MASK_TIF) as src:
    protected_mask = src.read(1)
    protected_excluded = (protected_mask == 1)
    
with rasterio.open(SETTLEMENT_MASK_TIF) as src:
    settlement_mask = src.read(1)
    settlement_excluded = (settlement_mask == 1)
    
with rasterio.open(WATER_MASK_TIF) as src:
    water_mask = src.read(1)
    water_excluded = (water_mask == 1)

print(f"Protected excluded: {protected_excluded.sum():,} cells ({protected_excluded.sum()/land_mask.sum()*100:.1f}%)")
print(f"Settlement excluded: {settlement_excluded.sum():,} cells ({settlement_excluded.sum()/land_mask.sum()*100:.1f}%)")
print(f"Water excluded: {water_excluded.sum():,} cells ({water_excluded.sum()/land_mask.sum()*100:.1f}%)")

# Step 3: Combine masks (Boolean OR)
print("\nStep 3: Combining masks (Boolean OR)")

combined_excluded = protected_excluded | settlement_excluded | water_excluded

excluded_cells = combined_excluded.sum()
allowed_cells = land_mask.sum() - excluded_cells

print(f"\nCombined result:")
print(f"  Total excluded: {excluded_cells:,} cells ({excluded_cells/land_mask.sum()*100:.1f}%)")
print(f"  Remaining allowed: {allowed_cells:,} cells ({allowed_cells/land_mask.sum()*100:.1f}%)")

# Step 4: Overlap analysis
print("\nStep 4: Overlap analysis")

protected_only = protected_excluded & ~settlement_excluded & ~water_excluded
settlement_only = settlement_excluded & ~protected_excluded & ~water_excluded
water_only = water_excluded & ~protected_excluded & ~settlement_excluded
protected_settlement = protected_excluded & settlement_excluded & ~water_excluded
protected_water = protected_excluded & water_excluded & ~settlement_excluded
settlement_water = settlement_excluded & water_excluded & ~protected_excluded
all_three = protected_excluded & settlement_excluded & water_excluded

print(f"  Protected only: {protected_only.sum():,} cells")
print(f"  Settlement only: {settlement_only.sum():,} cells")
print(f"  Water only: {water_only.sum():,} cells")
print(f"  Protected + Settlement: {protected_settlement.sum():,} cells")
print(f"  Protected + Water: {protected_water.sum():,} cells")
print(f"  Settlement + Water: {settlement_water.sum():,} cells")
print(f"  All three: {all_three.sum():,} cells")

overlap_total = (protected_settlement.sum() + protected_water.sum() + 
                 settlement_water.sum() + all_three.sum())
print(f"  Total overlap: {overlap_total:,} cells")

# Step 5: Create combined mask
print("\nStep 5: Creating combined mask")

combined_mask = np.zeros((rows, cols), dtype='int16')
combined_mask[combined_excluded] = 1
combined_mask[~land_mask] = NODATA_VALUE

print("Mask values:")
print(f"  0 = Allowed")
print(f"  1 = Excluded")
print(f"  -9999 = NoData (sea)")

# Step 6: Save combined mask
print("\nStep 6: Saving combined mask")

ref_meta.update({
    'dtype': 'int16',
    'nodata': NODATA_VALUE
})

with rasterio.open(COMBINED_MASK_TIF, 'w', **ref_meta) as dst:
    dst.write(combined_mask, 1)

print(f"Saved: {COMBINED_MASK_TIF}")

# Step 7: Verify
print("\nStep 7: Verifying alignment")
with rasterio.open(WLC_TIF) as ref, rasterio.open(COMBINED_MASK_TIF) as mask:
    checks = {
        'Shape': ref.shape == mask.shape,
        'Transform': ref.transform.almost_equals(mask.transform, precision=0.01),
        'CRS': ref.crs == mask.crs,
    }
    
    for check, result in checks.items():
        status = "MATCH" if result else "MISMATCH"
        print(f"  {check}: {status}")
    
    if all(checks.values()):
        print("\nPERFECT ALIGNMENT")

print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)

print(f"""
Summary:
  Individual constraints:
    Protected: {protected_excluded.sum():,} cells ({protected_excluded.sum()/land_mask.sum()*100:.1f}%)
    Settlement: {settlement_excluded.sum():,} cells ({settlement_excluded.sum()/land_mask.sum()*100:.1f}%)
    Water: {water_excluded.sum():,} cells ({water_excluded.sum()/land_mask.sum()*100:.1f}%)
  
  Combined (Boolean OR):
    Excluded: {excluded_cells:,} cells ({excluded_cells/land_mask.sum()*100:.1f}%)
    Allowed: {allowed_cells:,} cells ({allowed_cells/land_mask.sum()*100:.1f}%)
  
  Overlap: {overlap_total:,} cells
  
Output: {COMBINED_MASK_TIF}

Next step: Apply constraints to WLC
""")

COMBINE HARD CONSTRAINTS MASKS

Combining three constraint masks:
  1. Protected areas
  2. Settlement buffer (500m)
  3. Water bodies

Step 1: Loading reference grid
Grid: 235 x 131
Land cells: 9,743

Step 2: Loading constraint masks
Protected excluded: 2,652 cells (27.2%)
Settlement excluded: 5,885 cells (60.4%)
Water excluded: 70 cells (0.7%)

Step 3: Combining masks (Boolean OR)

Combined result:
  Total excluded: 7,144 cells (73.3%)
  Remaining allowed: 2,599 cells (26.7%)

Step 4: Overlap analysis
  Protected only: 1,215 cells
  Settlement only: 4,454 cells
  Water only: 25 cells
  Protected + Settlement: 1,405 cells
  Protected + Water: 19 cells
  Settlement + Water: 13 cells
  All three: 13 cells
  Total overlap: 1,450 cells

Step 5: Creating combined mask
Mask values:
  0 = Allowed
  1 = Excluded
  -9999 = NoData (sea)

Step 6: Saving combined mask
Saved: D:\BENV0093\outputs\hard_constraints_combined_mask.tif

Step 7: Verifying alignment
  Shape: MATCH
  Transform: MATCH
  CRS